In [66]:
#import libraries
import os
import numpy as np
import pandas as pd
import psycopg2

In [36]:
!ls

Customer Contracts$.csv
Pulse_2018_survey_public_data.csv
Pulse_2018_survey_public_data.numbers
Pulse_2018_survey_public_data_update.csv
Pulse_2018_survey_public_data_update.numbers
Pulse_2018_survey_public_data_updated.csv
inspect_2018.csv
pulse_2018_import.ipynb


In [37]:
#dataframe
# low_memory=False
df = pd.read_csv("Pulse_2018_survey_public_data.csv")
df.head()

,id,uasmembers,sampletype,batch,primary_respondent,hardware,language,start_date,start_year,start_month,...,q143,Scores_Spend,Scores_Save,Scores_Borrow,Scores_Plan,Scores_Total,FinancialHealth,base_weight,final_weight,imputation_flag
0,1,0,4 California,15 MSG 2017/11 California Batch 1,1 Primary respondent,0 None,1 English,30apr2018,2018,4,...,2 No,75.0,75.0,40.0,87.5,69.375,Coping,0.310482,0.249985,0.0
1,2,0,1 Nationally Representative,11 MSG 2016/04 Nat.Rep. Batch 7,1 Primary respondent,0 None,1 English,19jun2018,2018,6,...,2 No,87.5,62.5,72.5,57.5,70.000,Coping,0.285752,0.249923,0.0
2,3,1,1 Nationally Representative,8 MSG 2016/01 Nat.Rep. Batch 4,1 Primary respondent,0 None,1 English,06may2018,2018,5,...,1 Yes,62.5,87.5,90.0,82.5,80.625,Healthy,0.659296,0.249922,0.0
3,4,0,1 Nationally Representative,1 ASDE 2014/01 Nat.Rep.,1 Primary respondent,0 None,1 English,02may2018,2018,5,...,2 No,87.5,100.0,92.5,100.0,95.000,Healthy,0.325256,0.249986,0.0
4,5,2,1 Nationally Representative,8 MSG 2016/01 Nat.Rep. Batch 4,1 Primary respondent,0 None,1 English,01may2018,2018,5,...,2 No,67.5,100.0,82.5,70.0,80.000,Healthy,2.112595,0.937317,0.0


In [38]:
# clean table names
# lowercase, replace whitespace with _, replace symbols with _

file = "Pulse_2018_survey_public_data.csv"
clean_table_name = file.lower().replace(" ", "_").replace("?", "") \
                    .replace("-", "_").replace(r"/", "_").replace("\\", "_").replace("%", "") \
                    .replace(")", "").replace(r"(","").replace("$","")
clean_table_name

'pulse_2018_survey_public_data.csv'

In [43]:
#clean headers
# lowercase, remove whitespace, replace symbols with _

#first, get column names and list comprehension
df.columns
#df.dtypes gets the datatypes

id                      int64
uasmembers              int64
sampletype             object
batch                  object
primary_respondent     object
                       ...   
Scores_Total          float64
FinancialHealth        object
base_weight           float64
final_weight          float64
imputation_flag       float64
Length: 351, dtype: object

In [45]:
#create replacements dictionary
replacements = {
    'object' : 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]':'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [48]:
#create table pulse_2018_survey
#(
#    id int,
#    sampletype varchar,
#  batch varchar,
#    financialhealth varchar
#);

In [52]:
#create the create table sql statement - join name and datatype, replacing the pandas dtype with sql datatype as loop
col_string = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_string

'id int, uasmembers int, sampletype varchar, batch varchar, primary_respondent varchar, hardware varchar, language varchar, start_date varchar, start_year int, start_month int, start_day int, start_hour int, start_min int, start_sec int, end_date varchar, end_year varchar, end_month varchar, end_day varchar, end_hour varchar, end_min varchar, end_sec varchar, gender varchar, dateofbirth_year varchar, age varchar, agerange varchar, citizenus varchar, bornus varchar, stateborn varchar, countryborn varchar, countryborn_other varchar, immigrant_status varchar, statereside varchar, maritalstatus varchar, livewithpartner varchar, education varchar, hisplatino varchar, hisplatino_group varchar, white varchar, black varchar, nativeamer varchar, asian varchar, pacific varchar, race varchar, working varchar, sick_leave varchar, unemp_layoff varchar, unemp_look varchar, retired varchar, disabled varchar, lf_other varchar, laborstatus varchar, employmenttype varchar, workfullpart varchar, hourswor

In [60]:
#open database connection to postgres using psycopg2
conn_string = "host='localhost' \
                dbname = 'FHN' \
                user='user' \
                password='password'"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('opened database successfully')

opened database successfully


In [61]:
#drop tables with same name
cursor.execute("drop table if exists pulse_2018_survey;")

In [62]:
#CREATE TABLE
cursor.execute("create table pulse_2018_survey \
(id int, uasmembers int, sampletype varchar, batch varchar, primary_respondent varchar, hardware varchar, language varchar, start_date varchar, start_year int, start_month int, start_day int, start_hour int, start_min int, start_sec int, end_date varchar, end_year varchar, end_month varchar, end_day varchar, end_hour varchar, end_min varchar, end_sec varchar, gender varchar, dateofbirth_year varchar, age varchar, agerange varchar, citizenus varchar, bornus varchar, stateborn varchar, countryborn varchar, countryborn_other varchar, immigrant_status varchar, statereside varchar, maritalstatus varchar, livewithpartner varchar, education varchar, hisplatino varchar, hisplatino_group varchar, white varchar, black varchar, nativeamer varchar, asian varchar, pacific varchar, race varchar, working varchar, sick_leave varchar, unemp_layoff varchar, unemp_look varchar, retired varchar, disabled varchar, lf_other varchar, laborstatus varchar, employmenttype varchar, workfullpart varchar, hourswork varchar, hhincome varchar, anyhhmember varchar, hhmembernumber varchar, hhmemberage_1 varchar, hhmemberage_2 varchar, hhmemberage_3 varchar, hhmemberage_4 varchar, hhmemberage_5 varchar, hhmemberage_6 varchar, hhmemberage_7 varchar, hhmemberage_8 varchar, hhmemberage_9 varchar, hhmemberage_10 varchar, hhmemberage_11 varchar, hhmemberage_12 varchar, hhmemberage_13 varchar, hhmemberage_14 varchar, hhmemberage_15 varchar, hhmembergen_1 varchar, hhmembergen_2 varchar, hhmembergen_3 varchar, hhmembergen_4 varchar, hhmembergen_5 varchar, hhmembergen_6 varchar, hhmembergen_7 varchar, hhmembergen_8 varchar, hhmembergen_9 varchar, hhmembergen_10 varchar, hhmembergen_11 varchar, hhmembergen_12 varchar, hhmembergen_13 varchar, hhmembergen_14 varchar, hhmembergen_15 varchar, hhmemberin_1 varchar, hhmemberin_2 varchar, hhmemberin_3 varchar, hhmemberin_4 varchar, hhmemberin_5 varchar, hhmemberin_6 varchar, hhmemberin_7 varchar, hhmemberin_8 varchar, hhmemberin_9 varchar, hhmemberin_10 varchar, hhmemberin_11 varchar, hhmemberin_12 varchar, hhmemberin_13 varchar, hhmemberin_14 varchar, hhmemberin_15 varchar, hhmemberrel_1 varchar, hhmemberrel_2 varchar, hhmemberrel_3 varchar, hhmemberrel_4 varchar, hhmemberrel_5 varchar, hhmemberrel_6 varchar, hhmemberrel_7 varchar, hhmemberrel_8 varchar, hhmemberrel_9 varchar, hhmemberrel_10 varchar, hhmemberrel_11 varchar, hhmemberrel_12 varchar, hhmemberrel_13 varchar, hhmemberrel_14 varchar, hhmemberrel_15 varchar, lastmyhh_date varchar, cs_001 varchar, q001 varchar, q002 varchar, q003 varchar, q004 varchar, q005 varchar, q006 varchar, q007 varchar, q008 varchar, q009 varchar, q010 varchar, q011 varchar, q012 varchar, q013 varchar, q014 varchar, q015 varchar, q016 varchar, q017 varchar, q018 varchar, q018_other varchar, q019 varchar, q020 varchar, q021 varchar, q022 varchar, q023 varchar, q024 varchar, q025 varchar, q026 varchar, q026_other varchar, q026s1 varchar, q026s2 varchar, q026s3 varchar, q026s4 varchar, q026s5 varchar, q026s6 varchar, q026s7 varchar, q026s8 varchar, q026s9 varchar, q026s10 varchar, q026s11 varchar, q027 varchar, q027a varchar, q027b varchar, q028 varchar, q029 varchar, q030 varchar, q031 varchar, q032 varchar, q033 varchar, q034 varchar, q034a varchar, q035 varchar, q035_other varchar, q036 varchar, q037 varchar, q037_other varchar, q038 varchar, q038_other varchar, q038s1 varchar, q038s2 varchar, q038s3 varchar, q038s4 varchar, q038s5 varchar, q038s6 varchar, q038s7 varchar, q038s8 varchar, q038s9 varchar, q038s10 varchar, q038s11 varchar, q038s12 varchar, q039 varchar, q040 varchar, q041 varchar, q041_other varchar, q042 varchar, q042_other varchar, q042s1 varchar, q042s2 varchar, q042s3 varchar, q042s4 varchar, q042s5 varchar, q042s6 varchar, q042s7 varchar, q042s8 varchar, q042s9 varchar, q043 varchar, q043_other varchar, q043s1 varchar, q043s2 varchar, q043s3 varchar, q043s4 varchar, q043s5 varchar, q043s6 varchar, q043s7 varchar, q043s8 varchar, q043s9 varchar, q044 varchar, q045 varchar, q046 varchar, q047 varchar, q048 varchar, q049 varchar, q050 varchar, q051 varchar, q052 varchar, q053 varchar, q053_other varchar, q054 varchar, q055 varchar, q056 varchar, q057 varchar, q058 varchar, q059 varchar, q060 varchar, q061 varchar, q062 varchar, q063 varchar, q064 varchar, q066 varchar, q067 varchar, q068 varchar, q070 varchar, q071 varchar, q072 varchar, q073 varchar, q074 varchar, q075 varchar, q076 varchar, q077 varchar, q078 varchar, q079 varchar, q080 varchar, q081 varchar, q082 varchar, q083 varchar, q084 varchar, q085 varchar, q085_other varchar, q086 varchar, q087 varchar, q088 varchar, q089 varchar, q090 varchar, q091 varchar, q092 varchar, q093 varchar, q094 varchar, q095 varchar, q096 varchar, q097 varchar, q098 varchar, q099 varchar, q100 varchar, q101 varchar, q102 varchar, q103 varchar, q104 varchar, q105 varchar, q105_other varchar, q106 varchar, q107 varchar, q108 varchar, q109 varchar, q110 varchar, q111 varchar, q112 varchar, q113 varchar, q114 varchar, q115 varchar, q116 varchar, q117 varchar, q117a varchar, q118 varchar, q119 varchar, q120 varchar, q120_other varchar, q121 varchar, q122 varchar, q123 varchar, q123_other varchar, q123s1 varchar, q123s2 varchar, q123s3 varchar, q123s4 varchar, q123s5 varchar, q123s6 varchar, q123s7 varchar, q123s8 varchar, q123s9 varchar, q123s10 varchar, q123s11 varchar, q123s12 varchar, q124 varchar, q125 varchar, q125_other varchar, q125s1 varchar, q125s2 varchar, q125s3 varchar, q125s4 varchar, q125s5 varchar, q125s6 varchar, q125s7 varchar, q125s8 varchar, q125s9 varchar, q125s10 varchar, q125s11 varchar, q126_1_ varchar, q126_2_ varchar, q126_3_ varchar, q126_other_1_ varchar, q126_other_2_ varchar, q126_other_3_ varchar, q133 varchar, q134 varchar, q134_other varchar, q135 varchar, q136 varchar, q137 varchar, q138 varchar, q139 varchar, q140 varchar, q141 varchar, q142 varchar, q143 varchar, Scores_Spend float, Scores_Save float, Scores_Borrow float, Scores_Plan float, Scores_Total float, FinancialHealth varchar, base_weight float, final_weight float, imputation_flag float)")

In [63]:
#insert values to table

#save df to csv
df.to_csv('pulse_2018_surveydb.csv', header=df.columns, index=False, encoding='utf-8')

#open csv files, save as an object
my_file = open('pulse_2018_surveydb.csv')
print('file opened in memory')

file opened in memory


In [64]:
#upload to db
SQL_STATEMENT = """
COPY pulse_2018_survey FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print('file copied to db')


file copied to db


In [65]:
conn.commit()
cursor.close
print('table pulse_2018_survey imported to db completed')

table pulse_2018_survey imported to db completed
